Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [1]:
SIZE = 3

Let's solve the $(s^2-1)$-puzzle

In [2]:
from gx_utils import *

In [3]:
import logging
from random import choice
from collections import defaultdict

logging.basicConfig(format="%(message)s", level=logging.INFO)

In [4]:
import numpy as np

In [5]:
class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

# Tree-Search Algorithms

In [6]:
def goal_test(state):
    return state == GOAL


GOAL = State(np.array(list(range(1, SIZE**2)) + [0]).reshape((SIZE, SIZE)))
GOAL

array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 0]])

In [7]:
# (R, C) -> UP / RIGHT / DOWN / LEFT
MOVES = [np.array(_) for _ in [(-1, 0), (0, +1), (+1, 0), (0, -1)]]

In [8]:
def find_empty_space(board: np.ndarray):
    t = np.where(board == 0)
    return np.array([t[0][0], t[1][0]])


def is_valid(board: np.ndarray, action):
    return all(0 <= (find_empty_space(board) + action)[i] < board.shape[i] for i in [0, 1])


def possible_actions(state: State):
    return (m for m in MOVES if is_valid(state._data, m))

In [9]:
def result(state, action):
    board = state.copy_data()
    space = find_empty_space(board)
    pos = space + action
    board[space[0], space[1]] = board[pos[0], pos[1]]
    board[pos[0], pos[1]] = 0
    return State(board)

In [10]:
INITIAL_STATE = GOAL
for r in range(500):
    INITIAL_STATE = result(INITIAL_STATE, choice(list(possible_actions(INITIAL_STATE))))
INITIAL_STATE

array([[5, 7, 6],
       [3, 1, 2],
       [8, 4, 0]])

## Breadth-First

In [11]:
parent_state = dict()
state_depth = defaultdict(int)


def bf_search(initial_state: State):
    visited = set()
    frontier = PriorityQueue()
    state = initial_state
    parent_state[state], state_depth[state] = None, 0

    while state is not None and not goal_test(state):
        visited.add(state)
        for a in possible_actions(state):
            new_state = result(state, a)
            if new_state not in visited and new_state not in frontier:
                parent_state[new_state], state_depth[new_state] = state, state_depth[state] + 1
                frontier.push(new_state)
            elif new_state in frontier and state_depth[new_state] > state_depth[state] + 1:
                logging.debug(f"Update node in frontier: {state_depth[new_state]} to {state_depth[state]}")
                parent_state[new_state], state_depth[new_state] = state, state_depth[state] + 1
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    print(f"Found a solution in {len(path):,} steps; visited {len(visited):,} states")
    return list(reversed(path))

In [12]:
final = bf_search(INITIAL_STATE)

Found a solution in 29 steps; visited 128,034 states


## Depth-First

In [13]:
parent_state = dict()
state_depth = defaultdict(int)


def df_search(initial_state: State):
    visited = set()
    frontier = PriorityQueue()
    state = initial_state
    parent_state[state], state_depth[state] = None, 0

    n = 0
    while state is not None and not goal_test(state):
        visited.add(state)
        for a in possible_actions(state):
            new_state = result(state, a)
            if new_state not in visited and new_state not in frontier:
                parent_state[new_state], state_depth[new_state] = state, state_depth[state] + 1
                n -= 1
                frontier.push(new_state, p=n)
            elif new_state in frontier and state_depth[new_state] > state_depth[state] + 1:
                logging.debug(f"Update node in frontier: {state_depth[new_state]} to {state_depth[state]}")
                parent_state[new_state], state_depth[new_state] = state, state_depth[state] + 1
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    print(f"Found a solution in {len(path):,} steps; visited {len(visited):,} states")
    return list(reversed(path))

In [14]:
final = df_search(INITIAL_STATE)

Found a solution in 55,363 steps; visited 64,550 states


## Gready Best-First

In [15]:
def h(state):
    return np.sum((state._data != GOAL._data) & (state._data > 0))

In [16]:
parent_state = dict()
state_depth = defaultdict(int)


def gbf_search(initial_state: State):
    visited = set()
    frontier = PriorityQueue()
    state = initial_state
    parent_state[state], state_depth[state] = None, 0

    while state is not None and not goal_test(state):
        visited.add(state)
        for a in possible_actions(state):
            new_state = result(state, a)
            if new_state not in visited and new_state not in frontier:
                parent_state[new_state], state_depth[new_state] = state, state_depth[state] + 1
                frontier.push(new_state, p=h(new_state))
            elif new_state in frontier and state_depth[new_state] > state_depth[state] + 1:
                logging.debug(f"Update node in frontier: {state_depth[new_state]} to {state_depth[state]}")
                parent_state[new_state], state_depth[new_state] = state, state_depth[state] + 1
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    print(f"Found a solution in {len(path):,} steps; visited {len(visited):,} states")
    return list(reversed(path))

In [17]:
final = gbf_search(INITIAL_STATE)

Found a solution in 59 steps; visited 456 states


## A*

In [18]:
parent_state = dict()
state_depth = defaultdict(int)


def astar_search(initial_state: State):
    visited = set()
    frontier = PriorityQueue()
    state = initial_state
    parent_state[state], state_depth[state] = None, 0

    while state is not None and not goal_test(state):
        visited.add(state)
        for a in possible_actions(state):
            new_state = result(state, a)
            if new_state not in visited and new_state not in frontier:
                parent_state[new_state], state_depth[new_state] = state, state_depth[state] + 1
                frontier.push(new_state, p=state_depth[new_state] + h(new_state))
            elif new_state in frontier and state_depth[new_state] > state_depth[state] + 1:
                logging.debug(f"Update node in frontier: {state_depth[new_state]} to {state_depth[state]}")
                parent_state[new_state], state_depth[new_state] = state, state_depth[state] + 1
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data())
        s = parent_state[s]

    print(f"Found a solution in {len(path):,} steps; visited {len(visited):,} states")
    return list(reversed(path))

In [19]:
final = astar_search(INITIAL_STATE)

Found a solution in 29 steps; visited 55,901 states
